In [14]:
from transformers import AutoTokenizer, EsmForSequenceClassification
import torch

In [27]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D")
model = EsmForSequenceClassification.from_pretrained("facebook/esm2_t30_150M_UR50D")

tokenizer_config.json: 100%|██████████| 95.0/95.0 [00:00<00:00, 758kB/s]
vocab.txt: 100%|██████████| 93.0/93.0 [00:00<00:00, 832kB/s]
config.json: 100%|██████████| 779/779 [00:00<00:00, 6.84MB/s]
model.safetensors: 100%|██████████| 595M/595M [00:15<00:00, 38.3MB/s] 
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
inputs = tokenizer("MGSSHHHHHHSSGLVPRGSHMRGPNPTAASLEASAGPFTVRSFTVSRP", return_tensors="pt")

In [19]:
inputs

{'input_ids': tensor([[ 0, 20,  6,  8,  8, 21, 21, 21, 21, 21, 21,  8,  8,  6,  4,  7, 14, 10,
          6,  8, 21, 20, 10,  6, 14, 17, 14, 11,  5,  5,  8,  4,  9,  5,  8,  5,
          6, 14, 18, 11,  7, 10,  8, 18, 11,  7,  8, 10, 14,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [20]:
with torch.no_grad():
    logits = model(**inputs).logits

In [22]:
predicted_class_id = logits.argmax().item()

In [28]:
model2 = torch.load("esm2_t30_150M_UR50D.pt")

In [45]:
# Can be used
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t30_150M_UR50D")

Using cache found in /home/nikola_dev/.cache/torch/hub/facebookresearch_esm_main


In [41]:
import esm

In [81]:
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()

In [82]:
# model has a base class nn.Module
print(esm.model.esm2.ESM2.__bases__)
model

(<class 'torch.nn.modules.module.Module'>,)


ESM2(
  (embed_tokens): Embedding(33, 640, padding_idx=1)
  (layers): ModuleList(
    (0-29): 30 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=640, out_features=640, bias=True)
        (v_proj): Linear(in_features=640, out_features=640, bias=True)
        (q_proj): Linear(in_features=640, out_features=640, bias=True)
        (out_proj): Linear(in_features=640, out_features=640, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=640, out_features=2560, bias=True)
      (fc2): Linear(in_features=2560, out_features=640, bias=True)
      (final_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=600, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((640,), eps=1e-05, elementw

In [83]:
from torch.nn import Identity
model.contact_head = Identity()
model.emb_layer_norm_after = Identity()
model.lm_head = Identity()
model

ESM2(
  (embed_tokens): Embedding(33, 640, padding_idx=1)
  (layers): ModuleList(
    (0-29): 30 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=640, out_features=640, bias=True)
        (v_proj): Linear(in_features=640, out_features=640, bias=True)
        (q_proj): Linear(in_features=640, out_features=640, bias=True)
        (out_proj): Linear(in_features=640, out_features=640, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=640, out_features=2560, bias=True)
      (fc2): Linear(in_features=2560, out_features=640, bias=True)
      (final_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): Identity()
  (emb_layer_norm_after): Identity()
  (lm_head): Identity()
)

In [96]:
model.eval()
batch_converter = alphabet.get_batch_converter()
data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG")
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

In [99]:
with torch.no_grad():
    res = model(batch_tokens)

In [100]:
res['logits'].shape

torch.Size([2, 67, 640])